## Convert all netCDF NCAR timestep files to Zarr 512 arrays, with Grouped Velocity components, with (64,64,64) chunk size, round-robined across FileDB nodes (spatially using Z-order)

<font color="red">Old Dask version gives this error https://github.com/dask/distributed/issues/3955</font>

<font color='orange'>Note: Careful when Setting Dask `local_directory` to remote server (e.g. Temporary) will HUGELY slow down functions</font>

<font color='cyan'>Parallel version needs Large job</font>

<font color = 'gold'>TODO fix MemoryError: Unable to allocate 32.0 GiB for an array with shape (2048, 2048, 2048) and data type float32 when looping over multiple timesteps</font>

In [ ]:
desired_cube_side = 512
chunk_size = 64
raw_ncar_folder_path = '/home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt'
use_dask = True
dest_folder_name = "sabl2048b" # B is the high-rate data
write_type = "prod" # or "back" for backup

n_dask_workers = 4 # For Dask rechunking
num_threads = 34  # For writing to FileDB
dask_local_dir = '/home/idies/workspace/turb/data02_02'


timestep_nr = 2
# timestep_range = range(1) # This doesn't work with MemoryError: Unable to allocate 32.0 GiB for an array with shape (2048, 2048, 2048) and data type float32

In [ ]:
%cd /home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing

In [ ]:
import xarray as xr
from utils import write_tools
import dask
import os
import threading
import queue

### Get target Folder list

In [ ]:
folders=write_tools.list_fileDB_folders()

# Avoiding 7-2 and 9-2 - they're too full as of May 2023
folders.remove("/home/idies/workspace/turb/data09_02/zarr/")
folders.remove("/home/idies/workspace/turb/data07_02/zarr/")

for i in range(len(folders)):
    folders[i] += dest_folder_name + "_" + str(i + 1).zfill(2) + "_" + write_type + "/"

# folders[:5]

# Create top-level dirs

# for folder_path in folders:
#     os.makedirs(folder_path, exist_ok=False)

<font color="orange">Don't delete the CD cell!</font>

In [ ]:
%cd /home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt

In [ ]:
# for timestep_nr in timestep_range:
data_xr = xr.open_dataset(raw_ncar_folder_path + "/jhd." + str(timestep_nr).zfill(3) + ".nc")

# Group 3 velocity components together
# This fails with Dask bcs. of write permission error on SciServer Job
# Never use dask with remote location on this!!
merged_velocity = write_tools.merge_velocities(data_xr, dask_local_dir=dask_local_dir
                                               , chunk_size_base=chunk_size, use_dask=True, n_dask_workers=n_dask_workers)


# Unabbreviate 'e', 'p', 't' variable names
merged_velocity = merged_velocity.rename({'e': 'energy', 't': 'temperature', 'p': 'pressure'})


dims = [dim for dim in data_xr.dims]
dims.reverse() # use (nnz, nny, nnx) instead of (nnx, nny, nnz)

# Split 2048^3 into smaller 512^3 arrays
smaller_groups, _ = write_tools.split_zarr_group(merged_velocity, desired_cube_side, dims)

# Given up in favor of Ryan's node coloring technique
#     z_order = write_tools.morton_order_cube(cube_side=4)
node_assignments = write_tools.node_assignment(cube_side=4)


cubes = smaller_groups

encoding={
    "velocity": dict(chunks=(chunk_size, chunk_size, chunk_size, 3), compressor=None),
    "pressure": dict(chunks=(chunk_size, chunk_size, chunk_size, 1), compressor=None),
    "temperature": dict(chunks=(chunk_size, chunk_size, chunk_size, 1), compressor=None),
    "energy": dict(chunks=(chunk_size, chunk_size, chunk_size, 1), compressor=None)
}

print('Done preparing data. Starting to write...')

In [ ]:
def write_to_disk(q):
    while True:
        try:
            chunk, dest_groupname, encoding = q.get(timeout=10)  # Adjust timeout as necessary
            
            print(f"Starting write to {dest_groupname}...")
            chunk.to_zarr(store=dest_groupname, mode="w", encoding=encoding)
            print(f"Finished writing to {dest_groupname}.")
        except queue.Empty:
            break
        finally:
            q.task_done()

In [ ]:
q = queue.Queue()


# Populate the queue with tasks
for i in range(len(cubes)):
    for j in range(len(cubes[i])):
        for k in range(len(cubes[i][j])):
            filedb_index = node_assignments[i][j][k]
            current_array = cubes[i][j][k]

            chunk_nr = str(i * 16 + j * 4 + k + 1).zfill(2)

            dest_groupname = os.path.join(folders[filedb_index - 1], dest_folder_name + chunk_nr + "_" + str(timestep_nr).zfill(3) + ".zarr")
            q.put((current_array, dest_groupname, encoding))

In [ ]:
# Create threads and start them

threads = []
for _ in range(num_threads):
    t = threading.Thread(target=write_to_disk, args=(q,))
    t.start()
    threads.append(t)

# Wait for all tasks to be processed
q.join()

# Wait for all threads to finish
for t in threads:
    t.join()

This is a cool hack to restart the kernel!

!kill -9 `pgrep -f "kernel-.*.json"` && echo "Kernel restarted"